In [1]:
import pandas as pd

In [ ]:
electoral_ref = pd.read_excel('../../data/Electoral College Votes.xlsx')
data_2020 = pd.read_csv('../../data/F20_phase3_0717_target.csv')
data_historical = pd.read_csv("../../data/Data-Jul16/combined_jul16.csv", encoding = "ISO-8859-1")


In [ ]:
### 1) create a new df with StateFips/Count

In [ ]:
state_fips_map = {}
for index, row in data_historical.iterrows():
    #print(row)
    state = row['STATE']
    #print(type(state))
    if (state_fips_map.get(state) == None):
        state_fips_map[state] = row['STATE_FIPS']
    
print(state_fips_map)

In [ ]:
state_fips_df = pd.DataFrame(list(state_fips_map.items()), columns=['STATE','STATE_FIPS'])

In [ ]:
state_fips_df.head()

In [ ]:
df = electoral_ref.merge(state_fips_df, left_on='STATE', right_on='STATE')

In [ ]:
df.head()

In [ ]:
data_2020.loc[data_2020['STATE_FIPS'] == 13]

In [ ]:
data_2020.drop('Unnamed: 0', inplace=True, axis=1)

In [ ]:
## 2) 

# create map of visited county_state
# create map of D electoral votes, R electoral votes
# loop over data_2020
# updaste the two maps accordingly. 

In [ ]:
def sum_of(x):
    d = 0
    r = 0
    
    for c in x:
        if (c == 'D'):
            d += 1
        else:
            r += 1
    return d, r

In [ ]:
"""extract how counties prediction for every state"""
visited = {}
electoral_votes = {'D':0, 'R':0 }
for idnex, row in data_2020.iterrows():
    state = int(row['STATE_FIPS'])
    
    rep_votes = int(row['REP_VOTES'])
    dem_votes = int(row['DEM_VOTES'])

    if (visited.get(state) == None):
        electoral_votes = {'D': dem_votes, 'R': rep_votes }
        visited[state] = electoral_votes
    else:
        #update existing map
        current_votes = visited.get(state)
        current_r = current_votes.get('R')
        current_d = current_votes.get('D')
        electoral_votes = {'D': current_d + dem_votes, 'R': current_r + rep_votes }
        visited[state] = electoral_votes

In [ ]:
## new
visited

In [ ]:
"""determine the winner of every state"""
state_predictions = {}
d_count = 0
r_count = 0

for key, val in visited.items():
    total_r = val.get('R')
    total_d = val.get('D')
    winner = 'D' if total_d > total_r else 'R'
    
    if (winner == 'D'):
        d_count +=1 
    elif (winner == 'R'):
        r_count +=1
    
    state_predictions[key] = winner



In [ ]:
print('r=' + str(r_count))
print('d=' + str(d_count))

In [ ]:
##new
state_predictions

In [ ]:
df

In [ ]:
party_electoral = {}
for index, row in df.iterrows():
    state = int(row['STATE_FIPS'])
    votes = int(row['COUNT'])
    party = state_predictions.get(state)
    
    if (party_electoral.get(party) == None):
        party_electoral[party] = votes
    else:
        new_total = party_electoral.get(party) + votes
        party_electoral[party] = new_total
    

In [ ]:
party_electoral

In [ ]:
party_electoral